## This notebook describes the methodology behind the binary essentiality calls in Genome-wide CRISPR Screens Using Isogenic Cells Reveal Vulnerabilities Conferred by Loss of Tumor Suppressors manuscript by Feng et al.

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import scipy.stats as stats
import seaborn as sns

In [3]:
master_bf= pd.read_csv("BF_master_Xu_Feng_Tm_sup_screens_042320.txt", sep="\t", index_col=0)
master_bf.head()

,293A_WT_T22_AB_XF498,293A_LKB1_T22_AB,293A_PTEN_T22_AB,293A_VHL_T22_AB,293A_BAP1NUMBER2_16_T25_AB,293A_CDH1NUMBER2_15_T24_AB,293A_NF2NUMBER2_3_T24_AB,293A_WT_T21_AB_XF646,293A_PBRM1_T25_AB,293A_SETD2_T24_AB,293A_WT_T20_AB_XF804,293A_ARID1A_T21_AB,293A_NF1_T24_AB,293A_RB1_T21_AB,293A_TP53_T21_AB,293A_WT_T21_AB_XF821,293A_KEAP1_T22_AB,293A_g53BP1#1_T22_AB
GENE,,,,,,,,,,,,,,,,,,
A1BG,-3.976,0.325,-2.268,-13.424,-1.978,-6.636,-13.668,2.672,-4.049,-3.721,-10.339,-12.501,-1.496,2.411,-1.157,0.458,5.730,-16.922
A1CF,-0.221,4.165,1.068,11.453,-1.936,2.201,-7.592,1.028,-0.613,5.647,5.029,-0.509,2.955,2.438,-0.042,6.246,-4.169,4.336
A2M,11.255,13.856,2.643,14.523,9.193,3.044,-4.289,4.046,-1.074,4.912,4.074,4.976,0.018,12.320,10.420,-3.468,15.636,0.137
A2ML1,-10.607,-5.756,-15.726,-10.805,-17.877,-10.600,-13.521,-14.324,-11.535,-13.244,-2.162,-14.916,-5.025,-5.920,-11.450,-3.412,-4.492,-5.172
A3GALT2,-7.844,-3.118,-2.142,-3.376,-2.967,-2.774,-9.805,-9.340,-7.648,-8.573,-10.568,-9.185,-5.970,-14.048,-8.173,-5.865,-11.538,-5.509


In [4]:
master_bf.shape

(18053, 18)

### We used the binwise FDR values identified with the method described in : binwide FDR values for kidney cells from saturation modeling approach described in M. Dede, E. Kim, T. Hart, Biases and blind-spots in genome-wide CRISPR knockout screens. bioRxiv, (2020): https://www.biorxiv.org/content/10.1101/2020.01.16.909606v1. The log prior ration was capped at 5 to prevent instability.

In [5]:
prior_ratio_log2=dict()
for i in arange(1,18):
    prior_ratio_log2[i]=5
prior_ratio_log2[1]=0
prior_ratio_log2[2]=2
prior_ratio_log2[3]=3.6


In [6]:
master_gene_bf_dict=dict()
for i in master_bf.index:
    gene_df=pd.DataFrame(index=master_bf.loc[i].sort_values(ascending=False).index, columns=['BF_BAGEL'])
    gene_df['BF_BAGEL']=master_bf.loc[i].sort_values(ascending=False).values
    master_gene_bf_dict[i]=gene_df

In [7]:
initial_BF_threshold=10  
initial_prior= -3 # initial prior based on the empirical information from genome-wide CRISPR-Cas9 screens: background 
                  # expectation of gene essentiality in humans as 10%, which yields a log prior ratio ~ -3.
log2_odds= 7
binary_call_dict=dict()
calc_log_prior_ratio=dict()
new_bf_threshold=dict()
bf_from_bagel=dict()


for m in master_bf.index.values:
    binary_call_dict[m]=dict()
    
    for i in arange(len(master_gene_bf_dict[m].index)):

        if i ==0: # for the first screen 

            calc_log_prior_ratio[i]= initial_prior # we don't use the binwise FDR since it is the first observation
            bf_from_bagel[m]=dict()
            bf_from_bagel[m][i]=master_gene_bf_dict[m].iloc[i].values

            if bf_from_bagel[m][i]>= initial_BF_threshold:
                binary_call=1
            else:
                binary_call=0

            new_bf_threshold[i]=initial_BF_threshold
        else:
            calc_log_prior_ratio[i]= prior_ratio_log2[i] # get the calculated log2 prior ratio
            bf_from_bagel[m][i]=master_gene_bf_dict[m].iloc[i].values

            new_bf_threshold[i]=  log2_odds - calc_log_prior_ratio[i]     #set new BF threshold= log2odds - prior



            if bf_from_bagel[m][i]>= new_bf_threshold[i]:

                binary_call=1
            else:
                binary_call=0

      
        binary_call_dict[m][master_gene_bf_dict[m].index[i]]=binary_call
       


In [8]:
binary_call_dict

{'A1BG': {'293A_KEAP1_T22_AB': 0,
  '293A_WT_T21_AB_XF646': 0,
  '293A_RB1_T21_AB': 0,
  '293A_WT_T21_AB_XF821': 0,
  '293A_LKB1_T22_AB': 0,
  '293A_TP53_T21_AB': 0,
  '293A_NF1_T24_AB': 0,
  '293A_BAP1NUMBER2_16_T25_AB': 0,
  '293A_PTEN_T22_AB': 0,
  '293A_SETD2_T24_AB': 0,
  '293A_WT_T22_AB_XF498': 0,
  '293A_PBRM1_T25_AB': 0,
  '293A_CDH1NUMBER2_15_T24_AB': 0,
  '293A_WT_T20_AB_XF804': 0,
  '293A_ARID1A_T21_AB': 0,
  '293A_VHL_T22_AB': 0,
  '293A_NF2NUMBER2_3_T24_AB': 0,
  '293A_g53BP1#1_T22_AB': 0},
 'A1CF': {'293A_VHL_T22_AB': 1,
  '293A_WT_T21_AB_XF821': 0,
  '293A_SETD2_T24_AB': 1,
  '293A_WT_T20_AB_XF804': 1,
  '293A_g53BP1#1_T22_AB': 1,
  '293A_LKB1_T22_AB': 1,
  '293A_NF1_T24_AB': 1,
  '293A_RB1_T21_AB': 1,
  '293A_CDH1NUMBER2_15_T24_AB': 1,
  '293A_PTEN_T22_AB': 0,
  '293A_WT_T21_AB_XF646': 0,
  '293A_TP53_T21_AB': 0,
  '293A_WT_T22_AB_XF498': 0,
  '293A_ARID1A_T21_AB': 0,
  '293A_PBRM1_T25_AB': 0,
  '293A_BAP1NUMBER2_16_T25_AB': 0,
  '293A_KEAP1_T22_AB': 0,
  '293A_NF2NUMBE

In [9]:
new_bf_threshold

{0: 10,
 1: 7,
 2: 5,
 3: 3.4,
 4: 2,
 5: 2,
 6: 2,
 7: 2,
 8: 2,
 9: 2,
 10: 2,
 11: 2,
 12: 2,
 13: 2,
 14: 2,
 15: 2,
 16: 2,
 17: 2}

In [10]:
master_table=pd.DataFrame(binary_call_dict).T
master_table.head()

,293A_KEAP1_T22_AB,293A_WT_T21_AB_XF646,293A_RB1_T21_AB,293A_WT_T21_AB_XF821,293A_LKB1_T22_AB,293A_TP53_T21_AB,293A_NF1_T24_AB,293A_BAP1NUMBER2_16_T25_AB,293A_PTEN_T22_AB,293A_SETD2_T24_AB,293A_WT_T22_AB_XF498,293A_PBRM1_T25_AB,293A_CDH1NUMBER2_15_T24_AB,293A_WT_T20_AB_XF804,293A_ARID1A_T21_AB,293A_VHL_T22_AB,293A_NF2NUMBER2_3_T24_AB,293A_g53BP1#1_T22_AB
A1BG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
A1CF,0,0,1,0,1,0,1,0,0,1,0,0,1,1,0,1,0,1
A2M,1,1,1,0,1,1,0,1,1,1,1,0,1,1,1,1,0,0
A2ML1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
A3GALT2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
master_table.to_csv("binary_calls.txt", sep="\t", index=True)